# Nebula Graph Indexing and Retrieval

知识图谱已经在各个领域全面开花，如教育、医疗、司法、金融等。相比于无结构文本或半结构化的知识存储方式，
知识图谱具备了更加清晰、全面和严谨的行业背景信息，从知识的完备性和专业性上来看更适合作为RAG系统的数据存储介质。


> [NebulaGraph](https://docs.nebula-graph.com.cn/3.6.0/1.introduction/1.what-is-nebula-graph/) 是一款开源的、分布式的、易扩展的原生图数据库，能够承载包含数千亿个点和数万亿条边的超大规模数据集，并且提供毫秒级查询。NebulaGraph使用nGQL命令来查询图数据库
> [nGQL](https://docs.nebula-graph.com.cn/3.6.0/3.ngql-guide/1.nGQL-overview/1.overview/)是 NebulaGraph 使用的的声明式图查询语言，支持灵活高效的图模式，而且 nGQL 是为开发和运维人员设计的类 SQL 查询语言，易于学习。

本示例将使用真实的行业图数据库，演示如何通过大模型将用户的问题转译成数据库查询命令，并将查询结果作为上下文用于增强最终结果的生成

## 安装部署

Nebula Graph官方提供了多种[服务部署方法](https://docs.nebula-graph.com.cn/3.6.0/4.deployment-and-installation/1.resource-preparations/)，用户可以根据自己的计算资源进行选择。

本示例选择通过TAR包的形式在本地安装服务

### 1. 安装Nebula Graph

```shell
# 下载指定版本的安装包
wget https://oss-cdn.nebula-graph.com.cn/package/3.6.0/nebula-graph-3.6.0.ubuntu2004.amd64.tar.gz

# 解压安装包
tar -xvzf nebula-graph-3.6.0.ubuntu2004.amd64.tar.gz -C ~/nebulagraph/

# 修改配置文件：将子目录etc中的文件nebula-graphd.conf.default、
# nebula-metad.conf.default和nebula-storaged.conf.default重命名，
# 删除.default，即可应用 NebulaGraph 的默认配置。

# 进入scripts目录
./nebula.service start all			# 启动服务
./nebula.service status all			# 查看服务状态

# 通过NebulaGraph Console 添加Storage主机
# 下载地址：https://docs.nebula-graph.com.cn/3.6.0/4.deployment-and-installation/connect-to-nebula-graph/
# 登录console
./nebula-console-linux-amd64-v3.6.0 -addr 127.0.0.1 -port 9669 -u root -p nebula
ADD HOSTS "127.0.0.1":9779;
exit
```

### 2. 安装NebulaGraph Studio

```shell
# 推荐安装生态工具nebula studio，后续创建&修改schema、查看数据详情会更加方便
wget https://oss-cdn.nebula-graph.com.cn/nebula-graph-studio/3.8.0/nebula-graph-studio-3.8.0.x86_64.tar.gz
tar -xvf nebula-graph-studio-3.8.0.x86_64.tar.gz

#启动服务
cd nebula-graph-studio
./server

# 在浏览器输入 http://{ip}:7001即可打开web界面
```

## 数据构建

本示例使用医疗知识图谱数据搭建图数据库，数据来自：https://github.com/liuhuanyong/QASystemOnMedicalKG

### 1. 创建Space & Schema

可以在NebulaGraph Studio上完成该步操作，也可以在nebula-console中通过命令行完成创建：
```shell 
./nebula-console-linux-amd64-v3.6.0 -addr 127.0.0.1 -port 9669 -u root -p nebula
# Create Space 
CREATE SPACE `MedicaKG` (partition_num = 10, replica_factor = 1, charset = utf8, collate = utf8_bin, vid_type = FIXED_STRING(32));
USE `MedicaKG`;

# Create Tag: 
CREATE TAG `check` ( `name` string NULL COMMENT "诊断检查项目名称") ttl_duration = 0, ttl_col = "", comment = "诊断检查项目";
CREATE TAG `department` ( `name` string NULL COMMENT "医疗科室名称") ttl_duration = 0, ttl_col = "", comment = "医疗科室";
CREATE TAG `disease` ( `name` string NULL COMMENT "疾病名称", `cause` string NULL COMMENT "疾病病因", `prevent` string NULL COMMENT "预防措施", `cure_lasttime` string NULL COMMENT "治疗周期", `cure_way` string NULL COMMENT "治疗方式", `cured_prob` string NULL COMMENT "治愈概率", `easy_get` string NULL COMMENT "疾病易感人群", `description` string NULL COMMENT "疾病描述") ttl_duration = 0, ttl_col = "", comment = "疾病";
CREATE TAG `drug` ( `name` string NULL COMMENT "药品名称") ttl_duration = 0, ttl_col = "", comment = "药品";
CREATE TAG `food` ( `name` string NULL COMMENT "食物名称") ttl_duration = 0, ttl_col = "", comment = "食物";
CREATE TAG `producer` ( `name` string NULL COMMENT "在售药品名称") ttl_duration = 0, ttl_col = "", comment = "在售药品";
CREATE TAG `symptom` ( `name` string NULL COMMENT "疾病症状名称") ttl_duration = 0, ttl_col = "", comment = "疾病症状";

# Create Edge: 
CREATE EDGE `department_belongs_to` () ttl_duration = 0, ttl_col = "", comment = "医疗科室从属关系";
CREATE EDGE `disease_accompany_with` () ttl_duration = 0, ttl_col = "", comment = "疾病并发疾病";
CREATE EDGE `disease_common_drug` () ttl_duration = 0, ttl_col = "", comment = "疾病常用药品";
CREATE EDGE `disease_department` () ttl_duration = 0, ttl_col = "", comment = "疾病所属治疗科室";
CREATE EDGE `disease_do_eat` () ttl_duration = 0, ttl_col = "", comment = "疾病宜吃食物";
CREATE EDGE `disease_has_symptom` () ttl_duration = 0, ttl_col = "", comment = "疾病症状";
CREATE EDGE `disease_no_eat` () ttl_duration = 0, ttl_col = "", comment = "疾病忌吃食物";
CREATE EDGE `disease_recommend_drug` () ttl_duration = 0, ttl_col = "", comment = "疾病推荐药品";
CREATE EDGE `disease_recommend_eat` () ttl_duration = 0, ttl_col = "", comment = "疾病推荐食谱";
CREATE EDGE `drugs_of` () ttl_duration = 0, ttl_col = "", comment = "厂商－药物关系";

# Create Index: 
CREATE TAG INDEX `disease_index` ON `disease` ( `name`(50));
CREATE EDGE INDEX `department_belongs_to_index` ON `department_belongs_to` ();
CREATE EDGE INDEX `disease_accompany_with_index` ON `disease_accompany_with` ();
CREATE EDGE INDEX `disease_common_drug` ON `disease_common_drug` ();
CREATE EDGE INDEX `disease_department_index` ON `disease_department` ();
CREATE EDGE INDEX `disease_do_eat` ON `disease_do_eat` ();
CREATE EDGE INDEX `disease_has_symptom` ON `disease_has_symptom` ();
CREATE EDGE INDEX `disease_no_eat` ON `disease_no_eat` ();
CREATE EDGE INDEX `disease_recommend_eat` ON `disease_recommend_eat` ();
CREATE EDGE INDEX `disease_recomment_drug` ON `disease_recommend_drug` ();
CREATE EDGE INDEX `drugs_of_index` ON `drugs_of` ();
```

### 2. 导入数据

NebulaGraph 提供了多种导入数据的方法，本示例选择通过nebula3-python客户端连接图数据库并导入数据，代码可参考[脚本](import_data.py)
```shell
# 执行以下命令导入数据
python import_data.py --ip xxxxxxxxx --port xxxx --file_path xxxxxx --space xxxxxxx
```

## 查询数据库

In [ ]:
import os
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))

from rag.connector.knowledge_graph.nebula_graph import NebulaKnowledgeGraph
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate

openai_api_key = ""
llm = ChatOpenAI(
    model_name="gpt-4",
    openai_api_key=openai_api_key
)

# 建议根据实际的图数据对prompt进行优化
# 例如增加查询示例可以大幅提升nGQL命令的生成效率

NGQL_GENERATION_TEMPLATE = """<指令> 你是一位出色的AI助手，擅长将用户的问题转化成图数据库查询语句。
现在你的任务是根据用户输入的问题，生成nGQL命令，用于查询NebulaGraph图数据库。
只能使用提供的图数据节点、关系类型和属性信息，不允许使用除此之外的其它信息。</指令>
图结构: {schema} 

注意事项: 不要在回复中包含任何解释和抱歉信息。除了构建nGQL命令之外，请勿回答任何可能提出其他问题的问题。除了生成的nGQL命令外，请勿包含任何文本。
下面有几个例子，供你参考：

# 示例1：
问题: 支气管炎的预防措施有哪些？
nGQL命令: match (v:disease) where v.disease.name=="支气管炎" return v.disease.prevent;

# 示例2：
问题: 得了支气管炎可以吃哪些药？
nGQL命令: match (v1:disease)-[e:disease_recommend_drug]->(v2:drug) where v.disease.name=="支气管炎" return v2.drug.name;

问题: {question}
nGQL命令: """

NGQL_GENERATION_PROMPT = PromptTemplate(
    template=NGQL_GENERATION_TEMPLATE, input_variables=["schema", "question"]
)
nebula_graph = NebulaKnowledgeGraph(llm, ngql_prompt=NGQL_GENERATION_PROMPT)
print(nebula_graph.graph_schema)

question = "病毒性肠炎有哪些并发症？"
docs = nebula_graph.call(question)
print(docs)

## LLM生成

In [ ]:
import os
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))

from rag.chains.generate import GenerateChain

generate_chain = GenerateChain(llm)
for ans in generate_chain.chain(query=question, docs=docs, history=[]):
    print(ans)